# Проект компании «ГлавРосГосНефть»

Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений

Описание данных
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).


## 1. Загрузка и подготовка данных

In [2]:
import os
import pandas as pd
import numpy as np

from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
RND = np.random.RandomState(12345)
BOOTHSTR_SAMPLS = 1000
BUDGET = 10**10
PER_1K_BARRELL = 450000
N_SAMPLS = 500
WELLS_NUMBER = 200

In [4]:
reg0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
reg1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
reg2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')
regs = [reg0, reg1, reg2]

In [5]:
for reg in regs:
    print(reg.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory 

In [6]:
for reg in regs:
    print(reg.duplicated().sum())

0
0
0


In [7]:
for reg in regs:
    print(reg.isnull().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


In [8]:
for reg in regs:
    print(len(reg['id'].unique()))

99990
99996
99996


In [9]:
for i in range(3):
    regs[i] = regs[i].drop_duplicates(subset=['id']).reset_index(drop=True)
    regs[i] = regs[i].drop(['id'], axis=1)


In [10]:
reg0.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


В Регионе 0 видна корреляция между Product и F2, а также обратная корреляция между признаками F0 и F1

In [11]:
reg1.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


В Регионе 1 показана почти 100-процентная коррелляция у Product и F2

In [12]:
reg2.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


В Регионе 2 видна корреляция между Product и F2

## 2. Обучение и проверка модели

In [13]:
def make_predictictions (data):
    features = data.drop(['id', 'product'], axis = 1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=.25, random_state=RND)
    model = LinearRegression()
    model.fit(features_train, target_train)
    return pd.Series(model.predict(features_valid)), target_valid

In [14]:
predicted_1, target_1 = make_predictictions(reg0)
predicted_2, target_2 = make_predictictions(reg1)
predicted_3, target_3 = make_predictictions(reg2)

In [15]:
def show_rmse (answers, predictions):
    mse = mean_squared_error(answers, predictions)
    print('Средний запас сырья, тыс. баррелей:', predictions.mean())
    print('RMSE:', mse ** 0.5)

In [16]:
print('Регион 1')
show_rmse(target_1, predicted_1)

Регион 1
Средний запас сырья, тыс. баррелей: 92.59256778438038
RMSE: 37.5794217150813


In [17]:
print('Регион 2')
show_rmse(target_2, predicted_2)

Регион 2
Средний запас сырья, тыс. баррелей: 68.76995145799754
RMSE: 0.889736773768064


In [18]:
print('Регион 3')
show_rmse(target_3, predicted_3)

Регион 3
Средний запас сырья, тыс. баррелей: 95.087528122523
RMSE: 39.958042459521614


Самое маленькое значение RMSE вышло у Региона 1 

## 3. Подготовка к расчёту прибыли

In [19]:
BUDGET_FOR_ONE = BUDGET / WELLS_NUMBER
Min_mean_product = BUDGET_FOR_ONE / PER_1K_BARRELL

In [20]:
Min_mean_product

111.11111111111111

Минимальный объем сырья при котором разработка месторождения не будет убыточной

In [21]:
print("Средний истинный запас сырья в регионах, тыс. баррелей:")
print()
count = 0
targets = [target_1, target_2, target_3]
for target in targets:
    count += 1
    print('Регион ', count,': ',  target.mean())

Средний истинный запас сырья в регионах, тыс. баррелей:

Регион  1 :  92.07859674082927
Регион  2 :  68.77162424984647
Регион  3 :  94.74895871720241


Исследуя средний истинный запас сырья в регионах, разработка месторождений в любом регионе будет убыточной.

Выберем 500 скважин из которых отберем 200 лучших

In [22]:
def revenue(target, probabilities):
    target = pd.Series(target).reset_index(drop=True)
    probabilities = pd.Series(probabilities).reset_index(drop=True)
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:WELLS_NUMBER]
    mlrd = 1000000000
    return ((PER_1K_BARRELL * selected.sum()) - BUDGET) / mlrd

In [23]:
revenue(target_1, predicted_1)

3.3208260431398524

In [24]:
revenue(target_2, predicted_2)

2.4150866966815108

In [25]:
revenue(target_3, predicted_3)

2.539915945842947

Самый прибыльный регион - Регион 1, так как может принести прибыль больше трех миллиардов рублей

## 4. Расчёт прибыли и рисков

In [30]:
def bootstrap (target, predictions):
    values = []
    for i in range(BOOTHSTR_SAMPLS):
        target_subsample = target.reset_index(drop=True).sample(n=N_SAMPLS, replace=True, random_state=RND)
        probs_subsample = predictions[target_subsample.index]
        rev = revenue(target_subsample, probs_subsample)
        values.append(rev)
    risk = 0.025
    values = pd.Series(values)
    lower = values.quantile(risk)
    #confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())
    confidence_interval = (values.quantile(0.025), values.quantile(0.975))
    mean = values.mean()

    print("Средняя выручка:", mean)
    print("2.5%-квантиль:", lower)
    print("Доверительный интервал:", confidence_interval)
    print("Риск убытка:", (values < 0).mean())

In [31]:
bootstrap(target_1, predicted_1)

Средняя выручка: 0.39314909009217
2.5%-квантиль: -0.11479433039467574
Доверительный интервал: (-0.11479433039467574, 0.9476706285417498)
Риск убытка: 0.065


In [32]:
bootstrap(target_2, predicted_2)

Средняя выручка: 0.4581817322551864
2.5%-квантиль: 0.07097091474347683
Доверительный интервал: (0.07097091474347683, 0.8667897899690604)
Риск убытка: 0.01


In [33]:
bootstrap(target_3, predicted_3)

Средняя выручка: 0.34474683178672594
2.5%-квантиль: -0.13898863509022652
Доверительный интервал: (-0.13898863509022652, 0.8390772678308407)
Риск убытка: 0.102


## Вывод
Самым финансово безопасным регионом для разработки является Регион 2, так как имеет наименьшее число риска убытка.